# 올리브영 홈페이지에서 베스트 상품100 크롤링

In [51]:
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
import urllib.parse
import requests
import pandas as pd
import re

In [62]:
product_id_text=[]
brand_text=[]
product_text=[]
sale_text=[]
discount_text=[]
result_1={}

now = datetime.now() #파일이름 현 시간으로 저장하기

url = "http://www.oliveyoung.co.kr/store/main/getBestList.do"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'lxml')
items_100 = soup.select("div.prd_info")

for items in items_100:
    product_id = items.select_one('div.prd_name > a')['data-ref-goodsno']
    product_id_text.append(product_id)

for items in items_100:
    brand = items.select_one('span.tx_brand').text
    brand_text.append(brand)
    
for items in items_100:    
    product = items.select_one('p.tx_name').text
    product_text.append(product)
    
for items in items_100:
    sale = items.select_one('span.tx_num').text
    sale = int(sale.replace(',','').strip())
    sale_text.append(sale)
    
for items in items_100:
    discount = items.select_one('span.tx_cur').text
    discount = int(discount.replace('원','').replace(',','').replace('~','').strip())
    discount_text.append(discount)





#모든 리스트 딕셔너리형태로 저장
result_1 = {"product_id":product_id_text,"brand":brand_text,"product":product_text,"sale":sale_text,"discount":discount_text}
df_1 = pd.DataFrame(result_1)  #df로 변환
# 새로 만들 파일이름 지정
outputFileName = '%s-%s-%s  %s시 %s분 %s초 merging.xlsx' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
df_1.to_excel(RESULT_PATH+outputFileName,sheet_name='sheet1')   


In [18]:
df_1.head()

,brand,discount,product,sale
0,클리오,22400,클리오 프로아이팔레트,32000
1,피지오겔,25800,피지오겔 DMT 페이셜 크림 150ml 기획,39700
2,닥터자르트,48000,[닥터자르트] 원더 시카페어 크림 세트 (시카 진정크림 본품더블용량),48000
3,바이오더마,36120,바이오더마 센시비오 500+500ml 듀오팩,42500
4,라운드어라운드,13600,[온라인단독] 라운드어라운드 그린티 약산성 클렌징폼 더블기획,17000


In [52]:
import time, lxml



contents = soup.select('.prd_name')
   
id_urls = []
for content in contents:
    ids = content.select('a')[0]['data-ref-goodsno']
    ids = 'http://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={}'.format(ids)
    id_urls.append(ids)

def get_details(id_url):
    global df
    driver = webdriver.Chrome('C:/Users/snari/Downloads/chromedriver_win32/chromedriver.exe')
    driver.get(id_url)
    
    driver.find_element_by_xpath('//*[@id="reviewInfo"]/a').click()
    html = BeautifulSoup(driver.page_source, 'lxml')
    
    name = html.select('p.prd_name')[0].text
    
    star = html.select('p.num').text.replace('점','').strip()
    star = float(star)
    
    review = html.select('a.goods_reputation').text.replace(',','')
    review = int(review)
    
    df = df.append(Series([name, star, review], index=columns), ignore_index=True)
    return df


for id_url in id_urls:
    try: 
        df = get_details(id_url)
    except:
        continue
    time.sleep(3)

# 새로 만들 파일이름 지정
outputFileName = 'review.xlsx'
df_2.to_excel(RESULT_PATH+outputFileName, index=False)

In [70]:
def getProductInfo(product_id):
    goods_url = 'http://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo='
    response = requests.get(goods_url + product_id)
    soup = BeautifulSoup(response.text, 'lxml')
    
    reviews = soup.select_one('#reviewInfo span').text
    reviews = int(reviews.replace(',','').replace('(','').replace(')',''))
    
    return [reviews]

# raw라는 칼럼 만들어서 거기에 리뷰 수 집어넣고
df_1['raw'] = df_1['product_id'].apply(lambda x: getProductInfo(x))

# raw라는 칼럼으로부터 review 데이터 받고
df_1['review'] = df_1['raw'].apply(lambda x: x[0])
df_1 = df_1.drop('raw', axis=1) # 쓸모없어진 raw라는 칼럼 날려버리면 끝.

In [71]:
df_1.head()

,brand,discount,product,product_id,sale,review
0,클리오,22400,클리오 프로아이팔레트,A000000121581,32000,251
1,피지오겔,25800,피지오겔 DMT 페이셜 크림 150ml 기획,A000000106176,39700,2046
2,닥터자르트,48000,[닥터자르트] 원더 시카페어 크림 세트 (시카 진정크림 본품더블용량),A000000129573,48000,21
3,바이오더마,36120,바이오더마 센시비오 500+500ml 듀오팩,A000000013194,42500,1460
4,라운드어라운드,13600,[온라인단독] 라운드어라운드 그린티 약산성 클렌징폼 더블기획,A000000121438,17000,869


In [72]:
df_1.corr()

,discount,sale,review
discount,1.000000,0.967617,-0.199735
sale,0.967617,1.000000,-0.184622
review,-0.199735,-0.184622,1.000000


In [80]:
import numpy as np
import matplotlib.pyplot as plt

brands = pd.DataFrame(df_1['brand'].value_counts())
brands


,brand
닥터지,7
클리오,6
닥터자르트,4
아벤느,4
피지오겔,4
메디힐,4
구달,3
바이오더마,3
라운드어라운드,2
나트라케어,2


In [90]:
# 중복되지 않은 유일한 브랜드의 갯수 추출
brand_unique = pd.unique(df_1['brand'])
print('브랜드 수 :',len(brand_unique))
print(brand_unique)

브랜드 수 : 62
['클리오' '피지오겔' '닥터자르트' '바이오더마' '라운드어라운드' '마녀공장' '나인위시스' '닥터지' '에스쁘아' '구달'
 '맥' '올리브영' '그린몬스터' '웰라쥬' '보다나' '클럭' '페리페라' '아토팜' '드림웍스' '스와니코코' '메디큐브'
 '브라운' '아비브' '세타필' '아임프롬' '나트라케어' '보타닉힐보' '메디힐' '라운드랩' '토리든' '아벤느' '듀이셀'
 '더마클라센' '한율' '고려은단' '더마비' '솔가' '유세린' '3CE' '라로슈포제' '모레모' '리리코스 마린에너지'
 '헤드스파7' '보타닉힐보 아이디얼포맨' 'grn+' '착한팩토리' '투쿨포스쿨' '네이처메이드' '한스킨' '일리윤' '셀퓨전씨'
 '필립스' 'AHC' '아로마티카' '필리밀리' '메이크프렘' '겐조' '바닐라코' '레모나' '실크테라피' '세노비스'
 '에버비키니']
